In [1]:
import urllib
import os.path
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Utile
def random_binomial(shape, p=0.0, dtype=None, seed=None):
    """Returns a tensor with random binomial distribution of values.
    # Arguments
        shape: A tuple of integers, the shape of tensor to create.
        p: A float, `0. <= p <= 1`, probability of binomial distribution.
        dtype: String, dtype of returned tensor.
        seed: Integer, random seed.
    # Returns
        A tensor.
    """
    if dtype is None:
        dtype = 'float32'
    if seed is None:
        seed = np.random.randint(10e6)
    return tf.where(tf.random_uniform(shape, dtype=dtype, seed=seed) <= p,
                    tf.ones(shape, dtype=dtype),
                    tf.zeros(shape, dtype=dtype))



In [4]:

# LOAD DATA
file_name = 'mnist_test_seq.npy'
if not os.path.isfile(file_name):
    print("could not find moving mnist: download it..")
    url = 'http://www.cs.toronto.edu/~nitish/unsupervised_video/mnist_test_seq.npy'
    urllib.urlretrieve(url, file_name)
    print("download complete")
else :
    print ("Data Exist")
    
data = np.load(file_name)
print data.shape

x_dim = data.shape[2] * data.shape[3]
print x_dim

#Preprocess Data




Data Exist
(20, 10000, 64, 64)
4096


In [ ]:
#reshape Data

def reshape(data_in ,number_of_frame = 2):
    N, M, H, W = batch.shape
    n = M-number_of_frame+1
    data_out = np.zeros((n * N, number_of_frame, 64, 64), 'uint8')
    for j in range(N):
        for i in range(n):
            data_out[j*] = batch[j, i:i+n]

    

In [6]:
numpy_rng = np.random.RandomState(1)

numfac  = 512
nummap  = 256

# pretrainig first layer
input_x1 = tf.placeholder(tf.float32, [None, x_dim])
input_x2 = tf.placeholder(tf.float32, [None, x_dim])

U1 = tf.Variable(tf.random_normal(shape=(x_dim, numfac)) * 0.01)
V1 = tf.Variable(tf.random_normal(shape=(x_dim, numfac)) * 0.01)
W1 = tf.Variable(numpy_rng.uniform(low=-0.01, high=+0.01, size=( numfac, nummap)).astype('float32'))

bias_U1 = tf.Variable(np.zeros(numfac, dtype='float32'))
bias_V1 = tf.Variable(np.zeros(numfac, dtype='float32'))
bias_W1 = tf.Variable(np.zeros(nummap, dtype='float32'))
bias_U1_out = tf.Variable(np.zeros(x_dim, dtype='float32'))
bias_V1_out = tf.Variable(np.zeros(x_dim, dtype='float32'))
bias_W1_out = tf.Variable(np.zeros(numfac, dtype='float32'))

# m=sig(W(U*X1 . V*X2 ))
M1 =  tf.sigmoid(tf.matmul(tf.multiply(tf.matmul(input_x1,U1) + bias_U1,tf.matmul(input_x2,V1) + bias_V1), W1)+ bias_W1)

output_x1 = tf.matmul(tf.multiply(tf.matmul(M1,tf.transpose(W1)) + bias_W1_out,tf.matmul(input_x2,V1) + bias_V1),tf.transpose(U1))+ bias_U1_out
output_x2 = tf.matmul(tf.multiply(tf.matmul(M1,tf.transpose(W1)) + bias_W1_out,tf.matmul(input_x1,U1) + bias_U1), tf.transpose(V1))+ bias_V1_out

cost_1 = tf.nn.l2_loss(output_x1-input_x1) + tf.nn.l2_loss(output_x2-input_x2)
optimizer_1 = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost_1)
